In [47]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request, IPython
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
from sagemaker import get_execution_role
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker
from sagemaker.predictor import csv_serializer

In [9]:
if int(sagemaker.__version__.split('.')[0]) == 2:
    print("Installing previous SageMaker Version and restarting the kernel")
    !{sys.executable} -m pip install sagemaker==1.72.0
    IPython.Application.instance().kernel.do_shutdown(True)

else:
    print("Version is good")
    
role = get_execution_role()
sess = sagemaker.Session()
region = boto3.session.Session().region_name
print("Region = {}".format(region))
sm = boto3.Session().client('sagemaker')

Version is good
Region = us-east-1


In [10]:
# create a s3 bucket to store model related stuff
bucket_name = 'airplane-ticket-model'
s3 = boto3.resource('s3')
try:
    if  region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    else: 
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [15]:
# import dataset from s3 bucket
bucket='planetickets'
data_key = 'Cleaned_2018_Flights.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
data = pd.read_csv("Cleaned_2018_Flights.csv")

In [17]:
data = data.dropna()
data = data.drop(columns = ['Unnamed: 0', 'ItinID', 'MktID', 'MktCoupons', 'OriginWac', 'DestWac', 'ContiguousUSA', "Miles"])

# Y: PricePerTicket; X: all other variables
X = data.drop(columns = ['PricePerTicket'])
Y = data[['PricePerTicket']]

# Split training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [18]:
numerical_cols = ["NumTicketsOrdered"]
categorical_cols = ['Quarter', "Origin", "Dest", "AirlineCompany"] 

In [39]:
enc = OneHotEncoder(handle_unknown='ignore')
X_train_transformed = enc.fit_transform(X_train[categorical_cols]).toarray()

In [41]:
X_train_transformed.shape

(190644, 316)

In [56]:
categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(
    transformers=[
            ('num', numerical_transformer, numerical_cols)
            ('cat', categorical_transformer, categorical_cols)], remainder='passthrough')

NameError: name 'numerical_transformer' is not defined

In [52]:
X_train_transformed = preprocessor.fit_transform(X_train).toarray()

In [55]:
X_train_transformed[0].sum()

4.0